In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
#
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#
# Larger scale for plots in notebooks
sns.set_context('notebook')
#
# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)


In [145]:
gadm = gpd.read_file('gadm41_GHA_shp/gadm41_GHA_0.shp')
districts = gpd.read_file('data/Ghana_New_260_District.zip')

In [ ]:
districts.plot()

In [ ]:
districts.info()

In [25]:
# districts_json = gpd.GeoSeries(districts["geometry"]).__geo_interface__


In [32]:
districts.head()

,FID,OBJECTID,REGION,DISTRICT,FID_,Shape_Leng,Shape_Area,Shape__Are,Shape__Len,geometry
0,1,1,GREATER ACCRA,ADENTA MUNICIPAL,0,51531.321934,7.925971e+07,7.925971e+07,51531.321934,"MULTIPOLYGON (((-0.09740 5.68393, -0.09787 5.6..."
1,2,2,GREATER ACCRA,LEDZOKUKU MUNICIPAL,1,26490.421694,2.972006e+07,2.972006e+07,26490.421694,"POLYGON ((-0.09551 5.58048, -0.09573 5.58023, ..."
2,3,3,NORTHERN EAST,CHEREPONI,2,266759.459907,1.428011e+09,1.428011e+09,266759.459907,"POLYGON ((0.38424 9.95148, 0.38456 9.95163, 0...."
3,4,4,UPPER WEST,WA EAST,3,387390.557156,4.460865e+09,4.460865e+09,387390.557156,"POLYGON ((-1.86006 10.37147, -1.85950 10.37077..."
4,5,5,UPPER WEST,WA MUNICIPAL,4,147841.186239,6.017038e+08,6.017038e+08,147841.186239,"POLYGON ((-2.41266 10.16492, -2.41226 10.16462..."


In [ ]:
import folium

m = folium.Map(location=[gadm.centroid.y, gadm.centroid.x], zoom_start=6, control_scale=True, tiles="CartoDB positron")
folium.Choropleth(
    geo_data=districts,
    name="choropleth",
    data=districts,
    key_on="feature.properties.FID",
    columns=["FID","Shape_Area"],
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.8,
    legend_name="Population"
).add_to(m)
m

In [5]:
# districts.groupby('REGION')['DISTRICT'].count()
type(districts['REGION'].unique())
regions = np.insert(districts['REGION'].unique(),0,'All')

numpy.ndarray

In [42]:
pop2010 = gpd.read_file('data/districtspopulation2010.min.json.zip')

In [43]:
pop2010.head()

,REGION,DISTRICT,District_C,DistCode,Pop_Total,Male,Female,Area_sq_km,Pop_Densit,Shape_STAr,Shape_STLe,geometry
0,GREATER ACCRA,LEDZOKUKU / KROWOR,Teshie,306,227932,109185,118747,47.5751,4790.993608,5.120938e+08,107394.647686,"POLYGON ((-0.14078 5.63457, -0.14044 5.63560, ..."
1,NORTHERN,CHEREPONI,Chereponi,817,53394,26206,27188,1374.7382,38.839395,1.479757e+10,858598.997531,"POLYGON ((0.38423 9.95152, 0.38455 9.95167, 0...."
2,UPPER WEST,WA MUNICIPAL,Wa,1002,107214,52996,54218,579.8588,184.896737,6.241551e+09,476260.261638,"POLYGON ((-2.41267 10.16497, -2.41227 10.16467..."
3,UPPER WEST,JIRAPA,Jirapa,1006,88402,41592,46810,1188.5692,74.376822,1.279366e+10,628861.006554,"POLYGON ((-2.43389 10.74119, -2.43205 10.74170..."
4,UPPER WEST,SISSALA WEST,Gwollu,1007,49573,24151,25422,2050.3671,24.177622,2.206998e+10,937781.777821,"POLYGON ((-2.45232 11.02336, -2.41042 11.01650..."


In [44]:
districts_old = gpd.read_file('data/DistrictsGhana.json.zip')

In [ ]:
districts_old.explore()

## Extracting Volta Lake shape from districts data

In [69]:
districts_old['dis'] = 1
districts['dis'] = 1
districts_old.head()

,Name,SHAPE_Leng,SHAPE_Area,geometry,dis
0,Nandom,122577.078142,5.361359e+08,"MULTIPOLYGON (((-2.90771 10.75599, -2.90738 10...",1
1,Lambussie Karni,171109.664479,8.689945e+08,"POLYGON ((-2.72587 10.71355, -2.71816 10.71359...",1
2,Sissala West,235793.098959,1.895255e+09,"POLYGON ((-2.42262 10.74178, -2.42907 10.74684...",1
3,Sissala East,376327.092088,5.352375e+09,"POLYGON ((-2.26611 10.54791, -2.26225 10.54792...",1
4,Kassena Nankana West,167412.148124,8.954016e+08,"POLYGON ((-1.49138 10.79952, -1.49505 10.81035...",1


In [ ]:
vlake_map = districts_old.dissolve(by="dis",)
vlake_overlay = districts.dissolve(by="dis")
ax = vlake_map.plot()

In [ ]:
vlake = vlake_overlay.overlay(vlake_map, how="difference")
vlake.plot()


### Cleaning the extracted shape
The extracted shape has slivers along the boundary of Ghana, using a bounding box to draw a tighter focus on the lake gives us the desired result of only the lake without any noise
bounding box data obtained from [geojson.io](geojson.io)

In [ ]:

vlake_bbox = gpd.read_file("data/volta_lake_bbox-min.json.zip")
vlake_geom = vlake_bbox.overlay(vlake, how="intersection")
vlake_geom.plot()

In [150]:
vlake_geom["name"] = "Volta Lake"
vlake_geom = vlake_geom[["name", "geometry"]]

vlake_proj = vlake_geom.to_crs(epsg=25000)
vlake_geom["area_sqkm"] = vlake_proj["geometry"].area/10**6
vlake_geom["length_km"] = vlake_proj["geometry"].length/1000

vlake_geom

c:\Users\Raymomd\miniconda3\envs\geospatial\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Raymomd\miniconda3\envs\geospatial\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,name,geometry,area_sqkm,length_km
0,Volta Lake,"POLYGON ((0.32018 6.77323, 0.31953 6.77316, 0....",7891.247407,4376.332224


In [96]:
# uncomment line below to save to file
# vlake_geom.to_file("data/volta_lake.json", driver="GeoJSON")

In [28]:
# InteractiveShell.ast_node_interactivity = 'last_expr'


In [8]:
districts.head()

,FID,OBJECTID,REGION,DISTRICT,FID_,Shape_Leng,Shape_Area,Shape__Are,Shape__Len,geometry
0,1,1,GREATER ACCRA,ADENTA MUNICIPAL,0,51531.321934,7.925971e+07,7.925971e+07,51531.321934,"MULTIPOLYGON (((-0.09740 5.68393, -0.09787 5.6..."
1,2,2,GREATER ACCRA,LEDZOKUKU MUNICIPAL,1,26490.421694,2.972006e+07,2.972006e+07,26490.421694,"POLYGON ((-0.09551 5.58048, -0.09573 5.58023, ..."
2,3,3,NORTHERN EAST,CHEREPONI,2,266759.459907,1.428011e+09,1.428011e+09,266759.459907,"POLYGON ((0.38424 9.95148, 0.38456 9.95163, 0...."
3,4,4,UPPER WEST,WA EAST,3,387390.557156,4.460865e+09,4.460865e+09,387390.557156,"POLYGON ((-1.86006 10.37147, -1.85950 10.37077..."
4,5,5,UPPER WEST,WA MUNICIPAL,4,147841.186239,6.017038e+08,6.017038e+08,147841.186239,"POLYGON ((-2.41266 10.16492, -2.41226 10.16462..."


## Dissolving district polygons into regional polygons

In [151]:
regions = (districts[["REGION","geometry","DISTRICT","Shape_Area","Shape_Leng"]]
           .dissolve(by="REGION", aggfunc={
                         "DISTRICT":"count",
                        "Shape_Area":"sum",
                         "Shape_Leng":"sum"
                         }))
regions =regions.rename(index={"NORTHERN EAST":"NORTH EAST"})

In [ ]:
regions.head()

In [ ]:
regions.explore()

## Merging regional population data with regional geometry

In [20]:
regional_population = pd.read_json("population2021.json",orient="records")


,region,population
0,Ashanti,"5,440,463"
1,Bono,"1,208,649"
2,Greater Accra,"5,455,692"
3,Central,"2,859,821"
4,Eastern,"2,925,653"
5,Northern,"2,310,939"
6,Western,"2,060,585"
7,Upper East,"1,301,226"
8,Upper West,"901,502"
9,Volta,"1,659,040"


In [99]:
regional_population["region"] = regional_population["region"].str.upper()
regional_population

,region,population
0,ASHANTI,"5,440,463"
1,BONO,"1,208,649"
2,GREATER ACCRA,"5,455,692"
3,CENTRAL,"2,859,821"
4,EASTERN,"2,925,653"
5,NORTHERN,"2,310,939"
6,WESTERN,"2,060,585"
7,UPPER EAST,"1,301,226"
8,UPPER WEST,"901,502"
9,VOLTA,"1,659,040"


In [135]:
regional_population = regional_population.assign(population = pd.to_numeric(regional_population['population'].str.replace(",","")))

In [136]:
regions_populations_geometry = regions.merge(regional_population, left_on = "REGION",right_on="region",how="left")

In [137]:
regions_populations_geometry.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   geometry     16 non-null     geometry
 1   DISTRICT     16 non-null     int64   
 2   Shape_Area   16 non-null     float64 
 3   Shape_Leng   16 non-null     float64 
 4   region       16 non-null     object  
 5   population   16 non-null     int64   
 6   pop_numeric  16 non-null     int64   
dtypes: float64(2), geometry(1), int64(3), object(1)
memory usage: 1.0+ KB


In [ ]:
regions_populations_geometry.explore()

### Adding attributes to the dataset

In [139]:
r_proj = regions_populations_geometry.to_crs(epsg=25000)

regions_populations_geometry = (regions_populations_geometry
                                .assign(
                                    area_sqkm=r_proj["geometry"].area/10**6, 
                                    length_km=r_proj["geometry"].length/1000))

regions_populations_geometry.head()

,geometry,DISTRICT,Shape_Area,Shape_Leng,region,population,pop_numeric,area_sqkm,length_km
0,"POLYGON ((-2.41612 6.65887, -2.41627 6.65893, ...",6,5.305807e+09,1.286558e+06,AHAFO,564668,564668,5195.627095,672.184744
1,"POLYGON ((-1.54638 5.93864, -1.54645 5.93833, ...",43,2.488943e+10,6.316295e+06,ASHANTI,5440463,5440463,24370.600314,1369.525556
2,"POLYGON ((-2.77780 7.02027, -2.77747 7.02015, ...",12,9.464266e+09,2.204576e+06,BONO,1208649,1208649,11650.588585,868.411939
3,"POLYGON ((-1.43358 7.57346, -1.43520 7.56992, ...",11,2.385816e+10,2.756013e+06,BONO EAST,1203400,1203400,23248.531020,1137.602798
4,"POLYGON ((-1.32407 5.09554, -1.32477 5.09518, ...",22,9.820103e+09,2.851173e+06,CENTRAL,2859821,2859821,9659.385164,885.350672


### Final dataset

In [141]:
regions_populations_geometry = (regions_populations_geometry
                                .rename(columns = {"DISTRICT" : "no_of_districts"})
                                .drop(["Shape_Area","Shape_Leng","pop_numeric"], axis = 1)
                                )

In [142]:
regions_populations_geometry

,geometry,no_of_districts,region,population,area_sqkm,length_km
0,"POLYGON ((-2.41612 6.65887, -2.41627 6.65893, ...",6,AHAFO,564668,5195.627095,672.184744
1,"POLYGON ((-1.54638 5.93864, -1.54645 5.93833, ...",43,ASHANTI,5440463,24370.600314,1369.525556
2,"POLYGON ((-2.77780 7.02027, -2.77747 7.02015, ...",12,BONO,1208649,11650.588585,868.411939
3,"POLYGON ((-1.43358 7.57346, -1.43520 7.56992, ...",11,BONO EAST,1203400,23248.531020,1137.602798
4,"POLYGON ((-1.32407 5.09554, -1.32477 5.09518, ...",22,CENTRAL,2859821,9659.385164,885.350672
5,"POLYGON ((-0.87897 5.75368, -0.87897 5.75220, ...",33,EASTERN,2925653,18957.770622,843.298909
6,"POLYGON ((-0.22130 5.52978, -0.22189 5.53001, ...",29,GREATER ACCRA,5455692,3698.447058,466.511325
7,"POLYGON ((-0.20276 9.10906, -0.20289 9.10920, ...",16,NORTHERN,2310939,24843.184564,1831.215960
8,"POLYGON ((0.38471 9.95167, 0.38485 9.95160, 0....",6,NORTH EAST,658946,9070.425825,993.189887
9,"POLYGON ((0.47694 7.29169, 0.47503 7.29132, 0....",8,OTI,747248,11066.278901,788.814954


In [144]:
# uncomment line below to export to file as a geojson doc
# regions_populations_geometry.to_file("data/regional_population2021.json",driver="GeoJSON")